In [6]:
import numpy as np
import pandas as pd

from scipy.io import loadmat
import matplotlib.pyplot as plt
from matplotlib import cm
import cmocean

import xarray as xr

from PIL import Image

from netCDF4 import Dataset

import numpy as np
import pandas as pd

from scipy.io import loadmat
import matplotlib.pyplot as plt
from matplotlib import cm
import cmocean

import xarray as xr

from nbformat import read

from PIL import Image
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from matplotlib.offsetbox import AnchoredText

from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER

import os

# Bathymetry Plots
OG code by: Sawyer Brand

In [2]:
def bathy(csig,lonmax,lonmin,latmax,latmin,floats,stations):
    Floats = np.loadtxt(floats) #here, I have created a new variable that loads the float txt file when used

    Lonny = Floats[:,2] #here, I have created a new variable that is made up of all the rows (:) in the third column (2)
    # of the float text file. The [:,2] means the indices that correspond will all the rows of the third column.
    Latty = Floats[:,1] #here, I have created a new variable that is made up of all the rows (:) in the second column (1)
    # of the float text file. The [:,2] means the indices that correspond will all the rows of the third column.

        
    etopo = xr.open_mfdataset('ETOPO2v2g_f4.nc') #this line reads in the netCDF file within the same directory that
    #contains the etopo data

    etopo.keys() #this line displays the names of the variables contained within the ETOPO file 
    #so you can edit the below code if you need to 

    x = etopo.x # In the ETOPO file I used to build this code, the Lon variable was stored as 'x'
    #so I renamed etopo.x to be x so I didn't have to type so much
    y = etopo.y # In the ETOPO file I used to build this code, the Lat variable was stored as 'y'
    z = etopo.z # In the ETOPO file I used to build this code, the Bathymetry variable was stored as 'z '
    
    #So this is where the boundaries that you set up above come into play. You want to zoom into the area the cruise
    # is taking place but you don't want to just set the boundaries of the plot because it will still be processing 
    # the entire dataset. So instead, you'll limit the dataset itself to the area you want

    Bathy = z[(y>=LatMin)&(y<=LatMax),(x>=LonMin)&(x<=LonMax)] 
    Lon = x[(x>=LonMin)&(x<=LonMax)]
    Lat = y[(y>=LatMin)&(y<=LatMax)]

    #In the above code, you've selected for the indices (which are the numbers that determine the location of the numbers
    # within a matrix or vector or whatever) that correspond to the longitude and latitude set you want. 
    
    
    #Load the Subantarctic Front file:
    saf = np.loadtxt('saf_orsi.txt')
    #the first column will be Longitude (-180,180) format, the second column will be Latitude
    saf_lon = saf[:,0]
    saf_lat = saf[:,1]

    #load the Pacific front file:
    pf = np.loadtxt('pf_orsi.txt')
    #the first column will be Longitude (-180,180) format, the second column will be Latitude
    pf_lon = pf[:,0]
    pf_lat = pf[:,1]

    #load the Pacific front file:
    stf = np.loadtxt('stf_orsi.txt')
    #the first column will be Longitude (-180,180) format, the second column will be Latitude
    stf_lon = stf[:,0]
    stf_lat = stf[:,1]

    #load the Pacific front file:
    sbdy = np.loadtxt('sbdy_orsi.txt')
    #the first column will be Longitude (-180,180) format, the second column will be Latitude
    sbdy_lon = sbdy[:,0]
    sbdy_lat = sbdy[:,1]

    #load the Pacific front file:
    saccf = np.loadtxt('saccf_orsi.txt')
    #the first column will be Longitude (-180,180) format, the second column will be Latitude
    saccf_lon = saccf[:,0]
    saccf_lat = saccf[:,1]
    
    F = loadmat('FebIce.mat')
    FebIce = F['IceConc']
    FLon = F['Lon']
    FLat = F['Lat']
    nlon = F['nlon']
    nlat = F['nlat']
    
    FebIce[FebIce == 0] = 'NaN'
    
    S = loadmat('SeptIce.mat')
    SIce = S['IceConc']
    SLon = S['Lon']
    SLat = S['Lat']
    
    SIce[SIce == 0] = 'NaN'
    
    plt.figure(1,(30,20))
    ax = plt.axes(projection=ccrs.PlateCarree())

    im = plt.contourf(Lon,Lat,Bathy,1000,cmap=cmocean.cm.topo)
    cbar = plt.colorbar(im)
    plt.clim(-8000,8000) #CAUTION editing this colorbar because it will change where the "land"  color starts
    #plt.contour(Lon,Lat,Bathy,100)
    

    plt.plot(Lonny,Latty,'k-')

    plt.plot(Lonny,Latty,'r.',markersize=30,markeredgecolor='green');
    plt.plot(saf_lon,saf_lat,c='0.45',label='SAF',linewidth=4)
    plt.plot(pf_lon,pf_lat,c='0.45',label='PF',linewidth=4)
    plt.plot(saccf_lon,saccf_lat,c='0.45',label='SACCF',linewidth=4)
    plt.plot(sbdy_lon,sbdy_lat,c='0.45',label='SBDY',linewidth=4)
    plt.plot(stf_lon,stf_lat,c='0.45',label='STF',linewidth=4)
    
    ax.add_feature(cfeature.LAND,facecolor='black')
    
    plt.scatter(FLon,FLat,c=FebIce,vmin=0.1,vmax=100,cmap='cool',marker='^',edgecolors='black')
    
    plt.scatter(FLon,FLat,c=FebIce,vmin=0.1,vmax=50,cmap='coolwarm',marker='s',edgecolors='black')
    
    plt.title('Bathymetry for '+csig,fontsize=30)
    

    plt.xlim([LonMin,LonMax])
    plt.ylim([LatMin,LatMax])
    
    gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,
                  linewidth=2, color='gray', alpha=0.5, linestyle='--')
    
    gl.xlabels_top = False
    gl.ylabels_left = False
    gl.xlines = False
    gl.xformatter = LONGITUDE_FORMATTER
    gl.yformatter = LATITUDE_FORMATTER
    gl.xlabel_style = {'size': 20, 'color': 'black'}
    gl.ylabel_style = {'size': 20, 'color': 'black'}

    plt.legend()
    
    plt.savefig(csig+'/'+csig+'_Bathy.png')
    Image.open(csig+'/'+csig+'_Bathy.png').save(csig+'/'+csig+'_Bathy.jpg','JPEG')